In [23]:
import tkinter as tk
from tkinter import messagebox
import openai

# Set your OpenAI API key
openai.api_key = 'YOUR_OPENAI_API_KEY'

# Function to get a response from OpenAI
def get_gpt_response(prompt):
    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=[{'role': 'user', 'content': prompt}]
    )
    return response['choices'][0]['message']['content']

class VanGoghApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Discover Vincent van Gogh")
        
        # Set the window to full-windowed (maximized window without full-screen mode)
        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()
        self.window_width = int(screen_width)
        self.window_height = int(screen_height)
        self.root.geometry(f"{self.window_width}x{self.window_height}")
        
        self.root.configure(bg="#f0f0f5")

        # Initialize main frame
        self.main_frame = tk.Frame(self.root, bg="#f0f0f5")
        self.main_frame.pack(fill="both", expand=True)

        # Set up a few cards with multiple questions
        self.cards = [
            {
                "title": "Young Vincent",
                "color": "#FFCCCC",
                "locked": False,
                "questions": [
                    {"question": "What is the place of birth of Vincent van Gogh?", "answer": "zundert"},
                    {"question": "In what year was Vincent van Gogh born?", "answer": "1853"},
                    {"question": "What is Vincent van Gogh's nationality?", "answer": "dutch"}
                ],
                "progress": 0  # Initial progress
            },
            {
                "title": "Looking for a direction",
                "color": "#CCFFCC",
                "locked": False,
                "questions": [
                    {"question": "In which year did Vincent start painting?", "answer": "1880"},
                    {"question": "Who influenced Vincent's early work?", "answer": "millet"}
                ],
                "progress": 0
            },
            {
                "title": "Looking for a direction",
                "color": "#CCFFCC",
                "locked": True,
                "questions": [
                    {"question": "In which year did Vincent start painting?", "answer": "1880"},
                    {"question": "Who influenced Vincent's early work?", "answer": "millet"}
                ],
                "progress": 0
            }
        ]

        # Display the cards in the main frame
        self.display_cards()

    def display_cards(self):
        # Clear the main frame
        for widget in self.main_frame.winfo_children():
            widget.destroy()

        # Create a frame for the cards and scrollable canvas
        canvas = tk.Canvas(self.main_frame, bg="#f0f0f5", highlightthickness=0)
        scrollbar = tk.Scrollbar(self.main_frame, orient="horizontal", command=canvas.xview)
        canvas.configure(xscrollcommand=scrollbar.set)
        
        scrollable_frame = tk.Frame(canvas, bg="#f0f0f5")
        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")

        scrollbar.pack(fill="x", side="bottom")
        canvas.pack(fill="both", expand=True)

        # Create the cards in the scrollable frame
        for idx, card_data in enumerate(self.cards):
            color = card_data["color"] if not card_data["locked"] else "#d3d3d3"  # Gray out if locked
            card = tk.Frame(scrollable_frame, width=250, height=350, bg=color, relief="ridge", bd=4)
            card.grid(row=0, column=idx, padx=20, pady=20)

            title_label = tk.Label(card, text=card_data["title"], bg=color, font=("Arial", 14, "bold"))
            title_label.pack(pady=10)

            progress_text = f"Progress: {card_data['progress']}%" if not card_data["locked"] else "🔒 Locked"
            progress_label = tk.Label(card, text=progress_text, bg=color, font=("Arial", 12))
            progress_label.pack(pady=10)

            if not card_data["locked"]:
                open_button = tk.Button(card, text="Open", command=lambda idx=idx: self.open_question_module(idx))
                open_button.pack(pady=20)

        scrollable_frame.update_idletasks()
        canvas.config(scrollregion=canvas.bbox("all"))

    def open_question_module(self, card_index):
        # Set current card and reset question index for this module
        self.current_card = self.cards[card_index]
        self.current_question_index = 0

        # Clear the main frame for new layout
        for widget in self.main_frame.winfo_children():
            widget.destroy()

        # Left frame for questions
        question_frame = tk.Frame(self.main_frame, bg="#FFCCCC", width=self.window_width * 0.5, height=self.window_height)
        question_frame.grid(row=0, column=0, sticky="nsew")
        self.main_frame.grid_columnconfigure(0, weight=1)
        self.main_frame.grid_rowconfigure(0, weight=1)

        # Right frame for chat
        chat_frame = tk.Frame(self.main_frame, bg="#CCFFCC", width=self.window_width * 0.5, height=self.window_height)
        chat_frame.grid(row=0, column=1, sticky="nsew")

        # Display question
        self.display_question(question_frame)

        # Chat area setup
        self.chat_log = tk.Text(chat_frame, wrap="word", state="disabled", width=40, height=20, bg="#CCFFCC")
        self.chat_log.pack(pady=10, padx=10)

        self.chat_entry = tk.Entry(chat_frame, width=50)
        self.chat_entry.pack(side="left", padx=10)

        chat_btn = tk.Button(chat_frame, text="Ask Vincent", command=self.ask_vincent)
        chat_btn.pack(side="right")

    def display_question(self, frame):
        # Get current question text
        question_text = self.current_card["questions"][self.current_question_index]["question"]

        # Display question label
        question_label = tk.Label(frame, text=question_text, font=("Arial", 14, "bold"), bg="#FFCCCC")
        question_label.pack(pady=20)

        # Entry for answer
        self.answer_entry = tk.Entry(frame, width=40)
        self.answer_entry.pack(pady=10)

        # Submit button
        submit_btn = tk.Button(frame, text="Submit Answer", command=self.check_answer)
        submit_btn.pack(pady=10)

    def check_answer(self):
        # Validate answer and update progress
        user_answer = self.answer_entry.get().strip().lower()
        correct_answer = self.current_card["questions"][self.current_question_index]["answer"]

        if user_answer == correct_answer:
            messagebox.showinfo("Correct!", "That's the right answer!")
            self.current_question_index += 1
            self.current_card["progress"] = int((self.current_question_index / len(self.current_card["questions"])) * 100)

            if self.current_question_index < len(self.current_card["questions"]):
                # Clear question and display next one
                for widget in self.main_frame.grid_slaves(0, 0):  # Target question frame
                    widget.destroy()
                self.display_question(self.main_frame.grid_slaves(0, 0)[0])
            else:
                messagebox.showinfo("Completed", "You have completed this module!")
                self.display_cards()
        else:
            messagebox.showerror("Incorrect", "That's not the correct answer. Try again!")

    def ask_vincent(self):
        # Chat interface
        question = self.chat_entry.get()
        if question:
            self.chat_entry.delete(0, tk.END)
            self.chat_log.config(state="normal")
            self.chat_log.insert(tk.END, f"You: {question}\n")
            response = get_gpt_response(f"You are Vincent van Gogh. {question}")
            self.chat_log.insert(tk.END, f"Vincent: {response}\n")
            self.chat_log.config(state="disabled")
            self.chat_log.see(tk.END)

# Initialize and run the app
root = tk.Tk()
app = VanGoghApp(root)
root.mainloop()
